In [1]:
import torchvision.models as models
import torch
import torch.nn as nn
import torch.optim as optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

batch_size = 4
learning_rate = 0.0001
num_epochs = 5


transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomRotation(20),
                                transforms.RandomVerticalFlip(),transforms.Resize((224,224)),
                                transforms.ToTensor()])

trainset = datasets.CIFAR100(root='./data', train=True, download=True,
                                        transform=transform)

testset = datasets.CIFAR100(root='./data', train=False, download=True, 
                                       transform=transform)


train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)


Files already downloaded and verified


In [2]:
torch.cuda.is_available()
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images[0].shape)

torch.Size([3, 224, 224])


In [3]:
vgg16 = models.vgg16_bn(pretrained=True)

resnet18 = models.resnet18(pretrained=True)

inception = models.inception_v3(pretrained=True)


Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.torch/models/vgg16_bn-6c64b313.pth
553507836it [00:27, 19920433.85it/s]
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.torch/models/resnet18-5c106cde.pth
46827520it [00:02, 16614416.61it/s]
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.torch/models/inception_v3_google-1a9a5a14.pth
108857766it [00:01, 78389799.56it/s]


In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
vgg16 = vgg16.to(device)

In [5]:

# Loss Function
Loss_function = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(vgg16.parameters(), lr=learning_rate)

vgg16.children

# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = Loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss %.4f'
                  % (epoch+1, num_epochs, i, len(train_loader), loss.item()))

Epoch [1/5], Iter [0/12500] Loss 9.7729
Epoch [1/5], Iter [100/12500] Loss 5.3520
Epoch [1/5], Iter [200/12500] Loss 4.7307
Epoch [1/5], Iter [300/12500] Loss 5.4555
Epoch [1/5], Iter [400/12500] Loss 4.9639
Epoch [1/5], Iter [500/12500] Loss 4.0750
Epoch [1/5], Iter [600/12500] Loss 4.7685
Epoch [1/5], Iter [700/12500] Loss 4.4075
Epoch [1/5], Iter [800/12500] Loss 4.4773
Epoch [1/5], Iter [900/12500] Loss 4.3723
Epoch [1/5], Iter [1000/12500] Loss 4.6358
Epoch [1/5], Iter [1100/12500] Loss 3.6691
Epoch [1/5], Iter [1200/12500] Loss 3.5593
Epoch [1/5], Iter [1300/12500] Loss 5.1254
Epoch [1/5], Iter [1400/12500] Loss 4.1564
Epoch [1/5], Iter [1500/12500] Loss 3.9458
Epoch [1/5], Iter [1600/12500] Loss 3.5899
Epoch [1/5], Iter [1700/12500] Loss 3.1420
Epoch [1/5], Iter [1800/12500] Loss 3.4606
Epoch [1/5], Iter [1900/12500] Loss 3.5236
Epoch [1/5], Iter [2000/12500] Loss 4.5900
Epoch [1/5], Iter [2100/12500] Loss 3.0508
Epoch [1/5], Iter [2200/12500] Loss 3.5370
Epoch [1/5], Iter [2300

In [7]:
vgg16.eval()

correct = 0
total = 0
for img, label in test_loader:
    images = Variable(img.cuda())
    labels = Variable(label.cuda())

    out = vgg16(images)
    _, predicted = torch.max(out.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(vgg16.state_dict(), 'model_nn_Adam.ckpt')

Test Accuracy of the model on the 10000 test images: 55.42 %


In [8]:
vgg16.eval()

correct = 0
total = 0
for img, label in test_loader:
    images = img.to(device)
    labels = label.to(device)

    out = vgg16(images)
    _, predicted = torch.max(out.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(vgg16.state_dict(), 'model_nn_Adam.ckpt')

Test Accuracy of the model on the 10000 test images: 55.22 %
